In [43]:
import pandas as pd
import numpy as np
import json
import os
import unidecode
import unicodedata as ud
import regex as re
import xml.etree.ElementTree

In [44]:
def remove_html(string):
    return ' '.join(xml.etree.ElementTree.fromstring(string).itertext())

In [45]:
topcv_r = pd.read_csv('data/topcv/csv/topcv_done_mappings_raw.csv')

In [46]:
list(topcv_r)

['url',
 'job_title',
 'company_name',
 'job_salary',
 'job_type',
 'job_yoe',
 'job_desc',
 'job_req',
 'job_level',
 'mod_addr',
 'modified_deadline',
 'mapped_industries',
 'mod_ind_index',
 'mod_ind_kw',
 'min_salary',
 'max_salary',
 'mod_level',
 'mod_city',
 'mod_min_year',
 'mod_max_year',
 'true_inds',
 'true_text_inds']

In [47]:
topcv_r = topcv_r[['job_title', 'company_name', 'mod_level', 'job_desc', 'job_req', 'mod_city', 'modified_deadline',
        'min_salary', 'max_salary', 'mod_min_year', 'mod_max_year', 'true_inds', 'true_text_inds']].copy()

### Mapping text cities

In [48]:
cities = pd.read_csv('data/topcv/csv/mapping-cities - cities.csv').set_index('id')
cities

,name,nameEn
id,,
1,An Giang,An Giang
2,Bà Rịa - Vũng Tàu,Ba Ria - Vung Tau
3,Bắc Giang,Bac Giang
4,Bắc Kạn,Bac Kan
5,Bạc Liêu,Bac Lieu
...,...,...
67,Trà Vinh,Tra Vinh
68,Tuyên Quang,Tuyen Quang
69,Vĩnh Long,Vinh Long


In [49]:
text_cities_ls = []
for ct_ls in topcv_r['mod_city'].str.split(', '):
    temp_text_cities_ls = []
    for ct in ct_ls:
        ct = int(ct)
        temp_text_cities_ls.append(cities['name'].loc[ct])
    text_cities_ls.append(', '.join(temp_text_cities_ls))

In [50]:
topcv_r['text_city'] = text_cities_ls

In [51]:
topcv_r

,job_title,company_name,mod_level,job_desc,job_req,mod_city,modified_deadline,min_salary,max_salary,mod_min_year,mod_max_year,true_inds,true_text_inds,text_city
0,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,Ngân Hàng TNHH MTV Shinhan Việt Nam,6,"<div class=""content-tab""><p><strong>Làm việc t...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",25,2023-06-30,10000000.0,30000000.0,0,0.0,"52, 33, 8","Tư vấn, Ngân hàng, Chăm sóc khách hàng",Hà Nội
1,Senior Game Designer,Công ty TNHH Gameloft,6,"<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",66,2023-06-15,NaN,NaN,2,NaN,21,IT Phần mềm,TP.HCM
2,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...",CHI NHÁNH NHÀ MÁY GẠO HOA SEN - CÔNG TY TNHH X...,6,"<div class=""content-tab""><p>- Nhận nhiệm vụ hà...","<div class=""content-tab""><p>- Tốt nghiệp từ Tr...",13,2023-06-09,7000000.0,10000000.0,0,1.0,"9, 55","Cơ khí, Vận hành máy - Bảo trì - Bảo dưỡng thi...",Cần Thơ
3,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,CÔNG TY TNHH MỘT THÀNH VIÊN MACHINERY & TOOLS ...,6,"<div class=""content-tab""><p>Tìm kiến nguồn khá...","<div class=""content-tab""><p>Tốt nghiệp chuyên ...",25,2023-06-09,10000000.0,NaN,2,NaN,"26, 65, 64","Kinh doanh, Hàng tiêu dùng, Hàng gia dụng",Hà Nội
4,Giảng Viên Ngành Công Nghệ Thông Tin,Trường Đại học Thành Đô,9,"<div class=""content-tab""><p>- Quản lý, tổ chức...","<div class=""content-tab""><p>- Thạc sĩ trở lên,...",25,2023-05-31,15000000.0,30000000.0,0,0.0,"15, 21","Giáo dục - Đào tạo, IT Phần mềm",Hà Nội
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30537,Nhân Viên Inbound Sales,Công ty TNHH Đầu tư và Du lịch 365,6,"<div class=""content-tab""><p>- Giao dịch với kh...","<div class=""content-tab""><p>- Có kinh nghiệm t...","25, 66",2023-06-30,9000000.0,NaN,3,NaN,26,Kinh doanh,"Hà Nội, TP.HCM"
30538,Điều Dưỡng Trưởng Bệnh Viện,Bệnh Viện Quốc Tế DoLife,3,"<div class=""content-tab""><p>- Tổ chức, chỉ đạo...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",25,2023-05-24,NaN,NaN,2,NaN,58,Y tế - Chăm sóc sức khỏe,Hà Nội
30539,Nhân Viên Lễ Tân,Công ty Cổ phần Dịch vụ Y tế SBB,6,"<div class=""content-tab""><p>- Đón tiếp khách h...","<div class=""content-tab""><p>- Nam/Nữ, độ tuổi ...",25,2023-05-24,7000000.0,12000000.0,0,1.0,8,Chăm sóc khách hàng,Hà Nội
30540,Nhân Viên Đặt Lịch Hẹn Sự Kiện Mảng Du Lịch - ...,CÔNG TY CỔ PHẦN CÂU LẠC BỘ KỲ NGHỈ VIỆT NAM,6,"<div class=""content-tab""><ul><li>Tiếp cận với ...","<div class=""content-tab""><ul><li>Sử dụng được ...",25,2023-06-30,9000000.0,12000000.0,0,0.0,"26, 32, 25","Kinh doanh, Nghề nghiệp khác, Khách sạn - Nhà ...",Hà Nội


### Mapping text levels

In [52]:
levels = pd.read_csv("data/topcv/csv/mapping-levels - levels.csv").set_index('id')
levels

,name
id,
1,Tất cả cấp bậc
2,Quản lý cấp cao
3,Quản lý cấp trung
4,Quản lý nhóm - Giám sát
5,Chuyên gia
6,Chuyên viên - Nhân viên
7,Cộng tác viên
8,Mới Tốt Nghiệp
9,Thực tập sinh/Sinh viên


In [53]:
text_levels = []
for level in topcv_r['mod_level']:
    level = int(level)
    text_levels.append(levels['name'].loc[level])

In [54]:
len(text_levels)

30542

In [55]:
topcv_r['text_level'] = text_levels

In [56]:
topcv_t = topcv_r[['modified_deadline', 'job_title', 'true_inds', 'true_text_inds',
        'text_level', 'min_salary', 'max_salary', 'mod_min_year', 'mod_max_year',
        'text_city', 'job_desc', 'job_req']].copy()
topcv_t.columns = [
    'date', 'title', 'industry', 'mapped_industry', 'level', 'min_salary', 'max_salary',
    'min_year', 'max_year', 'city', 'job_desc', 'job_req'
]

In [57]:
topcv_t.to_csv('data/topcv/tidy_csv/topcv_tidy_raw.csv', index=False)

### Processing job description

In [58]:
topcv_t['mod_desc'] = topcv_t.job_desc.apply(remove_html)
topcv_t['mod_req'] = topcv_t.job_req.apply(remove_html)

In [59]:
print(topcv_t['mod_desc'][20])

Phát triển kinh doanh: Thực hiện chỉ tiêu bán hàng theo danh mục các sản phẩm dành cho KHDN Chủ động mở rộng tập khách hàng mới, khách hàng tiềm năng có chất lượng nhằm tư vấn và khai thác nhu cầu sử dụng các sản phẩm dịch vụ tại Bản Việt Phát triển mối quan hệ với khách hàng hiện hữu thông qua tối đa sử dụng bộ sản phẩm toàn diện, duy trì sự gắn kết và đáp ứng kịp thời nhu cầu của khách hàng Phân tích Báo cáo tài chính, Thẩm định, đề xuất cấp tín dụng và cơ cấu lại các hồ sơ cấp tín dụng cho khách hàng Tăng cường bán chéo sản phẩm Báo cáo công việc, cập nhật thông tin khách hàng đầy đủ và chính xác theo quy định Đảm bảo chất lượng dịch vụ Tuân thủ, kiểm soát rủi ro và tái đánh giá khách hàng Quản lý và hỗ trợ nhân viên nhằm đảm bảo an toàn, phòng tránh rủi ro lạm dụng


In [60]:
topcv_td = topcv_t.drop(['job_desc', 'job_req'], axis=1).copy()

In [61]:
topcv_td = topcv_td.rename(columns={'mod_desc':'description',
                        'mod_req':'requirements'}).copy()

In [62]:
topcv_td.to_csv('data/topcv/tidy_csv/topcv_tidy.csv', index=False)

In [63]:
pd.read_csv('data/topcv/tidy_csv/topcv_tidy.csv')

,date,title,industry,mapped_industry,level,min_salary,max_salary,min_year,max_year,city,description,requirements
0,2023-06-30,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,"52, 33, 8","Tư vấn, Ngân hàng, Chăm sóc khách hàng",Chuyên viên - Nhân viên,10000000.0,30000000.0,0,0.0,Hà Nội,Làm việc tại Văn phòng Shinhan Bank: Mục đích ...,"- Tốt nghiệp Đại học, cao đẳng hoặc trung cấp ..."
1,2023-06-15,Senior Game Designer,21,IT Phần mềm,Chuyên viên - Nhân viên,NaN,NaN,2,NaN,TP.HCM,Main responsibilities: Creating highly detaile...,Above 2 years of experience in a Game Designer...
2,2023-06-09,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...","9, 55","Cơ khí, Vận hành máy - Bảo trì - Bảo dưỡng thi...",Chuyên viên - Nhân viên,7000000.0,10000000.0,0,1.0,Cần Thơ,- Nhận nhiệm vụ hàng ngày hoặc hàng tuần từ Tr...,"- Tốt nghiệp từ Trung cấp trở lên, chuyên ngàn..."
3,2023-06-09,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,"26, 65, 64","Kinh doanh, Hàng tiêu dùng, Hàng gia dụng",Chuyên viên - Nhân viên,10000000.0,NaN,2,NaN,Hà Nội,"Tìm kiến nguồn khách hàng mới, có nhu cầu xây ...","Tốt nghiệp chuyên ngành cơ khí chế tạo, tự độn..."
4,2023-05-31,Giảng Viên Ngành Công Nghệ Thông Tin,"15, 21","Giáo dục - Đào tạo, IT Phần mềm",Thực tập sinh/Sinh viên,15000000.0,30000000.0,0,0.0,Hà Nội,"- Quản lý, tổ chức giảng dạy các môn học, học ...","- Thạc sĩ trở lên, đúng chuyên ngành - Có ưu t..."
...,...,...,...,...,...,...,...,...,...,...,...,...
30537,2023-06-30,Nhân Viên Inbound Sales,26,Kinh doanh,Chuyên viên - Nhân viên,9000000.0,NaN,3,NaN,"Hà Nội, TP.HCM","- Giao dịch với khách hàng, trả lời câu hỏi củ...",- Có kinh nghiệm trong lĩnh vực dịch vụ du lịc...
30538,2023-05-24,Điều Dưỡng Trưởng Bệnh Viện,58,Y tế - Chăm sóc sức khỏe,Quản lý cấp trung,NaN,NaN,2,NaN,Hà Nội,"- Tổ chức, chỉ đạo điều dưỡng, kỹ thuật viên c...",- Tốt nghiệp Đại học (Cử nhân Đại học Điều dưỡ...
30539,2023-05-24,Nhân Viên Lễ Tân,8,Chăm sóc khách hàng,Chuyên viên - Nhân viên,7000000.0,12000000.0,0,1.0,Hà Nội,"- Đón tiếp khách hàng, hướng dẫn khách hàng kh...","- Nam/Nữ, độ tuổi từ 22 - 30 - Trung t..."
30540,2023-06-30,Nhân Viên Đặt Lịch Hẹn Sự Kiện Mảng Du Lịch - ...,"26, 32, 25","Kinh doanh, Nghề nghiệp khác, Khách sạn - Nhà ...",Chuyên viên - Nhân viên,9000000.0,12000000.0,0,0.0,Hà Nội,Tiếp cận với tệp dữ liệu khách hàng có sẵn của...,"Sử dụng được các ứng dụng, phần mềm văn phòng ..."
